In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2hsv
from skimage import io
from scipy.ndimage.filters import gaussian_filter
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import cv2
from matplotlib import pyplot as plt

# Exercício sobre segmentação de imagens

Agora é a sua vez.

Utilizando o notebook anterior como ponto de partida, implemente uma função que receba a URL de uma foto de pessoa (de pé, de corpo inteiro e bem centralizada) e retorne a classificação dessa foto em 2 categorias: usando muita ou pouca roupa.

Sugestão: Comece pesquisando e montando uma base de dados com 10 exemplos de cada categoria. Salve as imagens no seu google drive para facilitar os testes. Para conseguir acessar, a partir do seu notebook, as imagens salvas no seu drive utilizando uma URL, siga [este tutorial](https://elfsight.com/blog/2018/02/how-to-upload-images-for-direct-url-in-google-drive/).

Exemplo de imagem:

![alt text](http://drive.google.com/uc?export=view&id=1hRdS38Yn9hvPmLI4awEnyVf7nobOP9n7)

Em seguida, implemente uma função auxiliar que calcula o percentual de pixels de pele em relação ao total de pixels da imagem. Use esse valor para decidir, por limiarização, a qual categoria uma imagem deve pertencer. 

Depois, utilize as 5 primeiras imagens de cada classe para efetuar testes e descobrir o melhor limiar de separação entre as classes.

Finalmente, implemente uma função auxliar para avaliar o desempenho de seu classificador nas 5 imagens restantes de cada categoria, computando o percentual de acerto. 

O que entregar: um notebook completo, contendo o código, comentários e o teste realizado sobre as imagens da base de dados armazenadas em seu google drive (com compartilhamento de link).




In [ ]:
# Retorna as informações de cada cor, como ocorrência e porcentagem
def getColorInformation(labels, clusters):
  
  # Contador de ocorrências de cada cor estimada (predita) pelo kmeans
  occurence_counter = Counter(labels)
  
  # Array de dicionários de cores
  colorInformation = []
 
  # Obtém a soma total de todas as ocorrências preditas
  totalOccurance = sum(occurence_counter.values()) 
  
  # Percorre todas as cores dominantes (mais frequentes) preditas
  for x in occurence_counter.most_common(len(clusters)):
    
    index = (int(x[0]))
    
    # Converte o número da cor em uma lista de 3 componentes (RGB)
    color = clusters[index].tolist()
    
    # Obtém a porcentagem da cada cor
    color_percentage= (x[1]/totalOccurance)
    
    # Constrói o dicionário que mapeia índices de cores em cores e porcentagens
    colorInfo = {"cluster_index":index , "color": color , "color_percentage" : color_percentage }
    
    # Acrescenta o dicionário à lista de dicionários de cores
    colorInformation.append(colorInfo)
    
  return colorInformation 

In [ ]:
# Extrai as cores dominande de uma imagem
def extractDominantColor(image,number_of_colors=2):
    
  # Cria uma cópia da imagem
  img = image.copy()
  
  # Reorganiza a imagem para que que seja um longo array unidimensional de RGBs
  img = img.reshape((img.shape[0]*img.shape[1]) , 3)
  
  #Inicializa o objeto KMeans, informando o número de clusters desejado
  clustering = KMeans(n_clusters=number_of_colors, random_state=0)
  
  # Realiza o agrupamento
  clustering.fit(img)
  
  # Obtém a informação das cores dominantes a partir dos rótulos e centróides dos clusters encontrados
  colorInformation = getColorInformation(clustering.labels_, clustering.cluster_centers_)
  
  return colorInformation

In [ ]:
# Retorna a imagem após a aplicação dos filtros
def getImageSkin(url):

  rgb_img = io.imread(url)
  width, height = rgb_img.shape[:2]
  hsv_img = rgb2hsv(rgb_img)
  hue_img = hsv_img[:, :, 0]
  sat_img = hsv_img[:, :, 1]
  value_img = hsv_img[:, :, 2]
  
  bin_img = (hue_img > 0/255) & (hue_img < 20/255) & (sat_img > 48/255) & \
   (sat_img < 255/255) & (value_img > 80/255) & (value_img < 255/255)
  
  skin_img = np.zeros((width,height,3), dtype="uint8")
  skin_img[:,:,0] = bin_img * 255 # for red
  skin_img[:,:,1] = bin_img * 255 # for green
  skin_img[:,:,2] = bin_img * 255 # for blue
  
  skin_img = gaussian_filter(skin_img,sigma=5)

  return skin_img


# Retorna a porcentagem de pele em uma imagem
def porcentagemSkin(url):
  # Resize image to a width of 250
  image = imutils.resize(getImageSkin(url),width=250)
  #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

  # Find the dominant color. Default is 1 , pass the parameter 'number_of_colors=N' where N is the specified number of colors 
  dominantColors = extractDominantColor(image, number_of_colors=2)

  return (dominantColors[1]['color_percentage'])


In [ ]:
# Definindo as listas de urls de testes para ajustes e para acurácia
links_test = [
             
"https://docs.google.com/uc?id=1LZwWYF0RCaIlT4W_5gcLXC1WL1Mbt-06",
"https://docs.google.com/uc?id=1uGXQBWHEailpisW4KZhnVWlympPsorfL",
"https://docs.google.com/uc?id=1_p3rTPubcc9F1H_Y16eXmzcgVXJPQV7u",
"https://docs.google.com/uc?id=1Zv-qGjZIl7bX569W93CuUTizoczfpX55",
"https://docs.google.com/uc?id=1MPJH39UftHaeJRomn6LBcaHMQjKv9uLV",
"https://docs.google.com/uc?id=1hIlM9j-3qpx4iUnL9ixEMD7pSJn3bZNd",
"https://docs.google.com/uc?id=1CMkQOD5ZBbUGOTBt6kwKCvFgFYRXIDs4",
"https://docs.google.com/uc?id=1JI-aYB_IXOEsqZYh1aTvtB8i1q6OkwOB",
'https://docs.google.com/uc?id=1NuhDJC13tE9MTYKfwkDxuoFxKM7rkaM3',
'https://docs.google.com/uc?id=1S3mLOboWohZ4Z0o139S4ENTwtwvDcRUS'

]
links_avaliar = [
("https://docs.google.com/uc?id=1gS--EJj6E5oo3f81foDQrUXCm33SVyHg","Muita Roupa"),
("https://docs.google.com/uc?id=1PDlul1R9R4TkhTO1v4wh4OKKSyS1mk79","Muita Roupa"),
("https://docs.google.com/uc?id=1_KAhf2bE_kRnmpTHASBAqqlkCvGUB8nv","Muita Roupa"),
("https://docs.google.com/uc?id=1ycpq0IgEaytVZD7pYGvNm4Xqkv7Kkwif","Muita Roupa"),
("https://docs.google.com/uc?id=1Yz6ujXf2QsQDJTqjtcGVS0BGVkr02zTa","Muita Roupa"),
("https://docs.google.com/uc?id=1Fm7ozz53YYS0V340W1jiBe-ezG74NCQh","Pouca Roupa"),
("https://docs.google.com/uc?id=1FBmogHjzWsiMXv2Wk-AuCqM-DNnXTwnT","Pouca Roupa"),
("https://docs.google.com/uc?id=1NeRoO7eYXtqJ8W3jb6V1oNYKitaNfQe9","Pouca Roupa"),
("https://docs.google.com/uc?id=1QAQImZ56TFYRu0Tk4LoF2HvRGmGvQkzm","Pouca Roupa"),
("https://docs.google.com/uc?id=18cHvYJcOgrW0BVVBwqTnEeMVbBDmjxIL","Pouca Roupa")

]

In [ ]:
# Função principal que recebe uma url de uma imagem e a classifica como Pouca Roupa ou Muita Roupa
def classificador(url):

  porcentagem_skin = porcentagemSkin(url)
  if(porcentagem_skin > 0.13):
    return "Pouca Roupa" 
  else:
    return "Muita Roupa"



In [ ]:
# Testes para encontrar o melhor limiar
for link in links_test:
  print(classificador(link))

Muita Roupa
Muita Roupa
Muita Roupa
Muita Roupa
Muita Roupa
Pouca Roupa
Pouca Roupa
Pouca Roupa
Pouca Roupa
Pouca Roupa


In [ ]:
def accuracy(lista):
  sum = 0.0
  for link in lista:
    if(classificador(link[0])== link[1]):
      sum = sum + 1
  return sum/len(lista)

In [ ]:
print(accuracy(links_avaliar))

0.9
